In [1]:
#importing the modules we need
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from xgboost import XGBClassifier
import numpy as np

In [2]:
#bag of words class, made during ml lab
class BagOfWords:
    def __init__(self):
        self.vocabulary = dict()
        self.words = []  
        
        
    def build_vocabulary(self, data):
        for sentence in data:
            for word in sentence:
                if word not in self.vocabulary:
                    self.vocabulary[word] = len(self.vocabulary)
                    self.words.append(word)
                    
            
    def get_features(self, data):
        features = np.zeros((len(data), len(self.vocabulary)))
        
        for id_sen, document in enumerate(data):
            for word in document:
                if word in self.vocabulary:
                    features[id_sen, self.vocabulary[word]] += 1
                    
        return features

In [3]:
#getting the data
def load_sample(file_name):
    f = open(file_name, 'r', encoding='utf8')
    
    indexes = []
    sentences = []
    
    for line in f.readlines():
        indexes.append(int("".join(line[:6])))
        sentences.append(line[7:].strip('\n').split())
        
    return indexes, sentences


def load_label(file_name):
    f = open(file_name, 'r', encoding='utf8')
    
    sentences = []
    
    for line in f.readlines():
        sentences.append(int(line[7]))
        
    return sentences

In [48]:
#train data
train_indexes, train_samples = load_sample("data/train_samples.txt")
train_labels = load_label("data/train_labels.txt")

#validation data
validation_indexes, validation_samples = load_sample("data/validation_samples.txt")
validation_labels = load_label("data/validation_labels.txt")

#test data
test_indexes, test_samples = load_sample("data/test_samples.txt")

In [49]:
bow = BagOfWords()
bow.build_vocabulary(train_samples + validation_samples)

train_features = bow.get_features(train_samples + validation_samples)
train_labels = train_labels + validation_labels

validation_features = bow.get_features(validation_samples)
test_features = bow.get_features(test_samples)

In [50]:
#train the model
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(train_features, train_labels)

MultinomialNB()

In [51]:
predicted_naive_bayes = naive_bayes_model.predict(test_features)

predicted = naive_bayes_model.predict(validation_features)
percentege_multinomial_nb = np.mean(predicted == validation_labels)
print(percentege_multinomial_nb)

0.9304


In [52]:
complement_model = ComplementNB()
complement_model.fit(train_features, train_labels)

ComplementNB()

In [53]:
predicted_complement_nb = complement_model.predict(test_features)

predicted = complement_model.predict(validation_features)
percentege_complement_nb = np.mean(predicted == validation_labels)
print(percentege_complement_nb)

0.9458


In [54]:
bernoulli_model = BernoulliNB()
bernoulli_model.fit(train_features, train_labels)

BernoulliNB()

In [55]:
predicted_bernoulli_nb = bernoulli_model.predict(test_features)

predicted = bernoulli_model.predict(validation_features)
percentege_bernoulli_nb = np.mean(predicted == validation_labels)
print(percentege_bernoulli_nb)

0.9084


In [12]:
#train the model
xgboost_model = XGBClassifier(learning_rate=0.1, gamma=1, reg_alpha=0.5, reg_lambda=0.5, max_depth=10, subsample=0.75, colsample_bytree=0.75)

train_labels_xgboost = train_labels
for i in range(len(train_labels_xgboost)):
    train_labels_xgboost[i] -= 1

xgboost_model.fit(train_features, train_labels)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.75,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=1, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0.5, ...)

In [56]:
predicted_xgboost = xgboost_model.predict(test_features)

predicted = xgboost_model.predict(validation_features)

for i in range(len(predicted)):
    predicted[i] += 1

percentege_xgboost = np.mean(predicted == validation_labels)
print(percentege_xgboost)

ValueError: Feature shape mismatch, expected: 52590, got 65227

In [58]:
predicted = []

for i in range(len(predicted_naive_bayes)):
    percentege = np.zeros(4)
    

    percentege[predicted_naive_bayes[i]] += percentege_multinomial_nb
    percentege[predicted_complement_nb[i]] += percentege_complement_nb
    percentege[predicted_bernoulli_nb[i]] += percentege_bernoulli_nb
    percentege[predicted_xgboost[i]] += percentege_xgboost

    predicted.append(np.argmax(percentege))

In [60]:
g = open("data/test_labels.txt", 'w')
g.write("id,label\n")

for idx in range(len(predicted)):
    g.write(f"{test_indexes[idx]},{predicted[idx]}\n")

In [62]:
print(predicted_naive_bayes[:10])

[2 2 2 1 2 1 1 2 2 2]
